# SHPC4001: Session 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## 1. Vectorization

An important issue when working with large arrays in NumPy is _vectorization_.

As a trivial example, let's compute squares of first 1000 non-negative integers.

In [ ]:
x = np.arange(1000, dtype=np.int64)

The "old school" approach would be to use a looping construct of some kind:

In [ ]:
y1 = np.zeros_like(x)

In [ ]:
%%timeit
for k, xk in enumerate(x):
    y1[k] = xk**2

In NumPy, we just square the original array directly:

In [ ]:
%%timeit
y2 = x**2

NOTE 1: as a rule, this kind of "loop slowness" is an overhead of interpreted languages like Python, and does not occur in compiled languages like C or Fortran.

NOTE 2: it may not always be obvious, but with some care it is often possible to rewrite algorithms written with loops in vectorized form. Doing so can result in dramatic speed gains in NumPy as the above example illustrates.

NOTE 3: in cases where vectorization is inconvenient or impossible, packages like [Cython](https://cython.org/) or [Numba](https://numba.pydata.org/) can be used to produce faster loops in Python. This should only be necessary in special cases, however.

## 2. Numerical precision exercises

**Exercise 1:** start with one, and add one-hundred-millionth one hundred million times

We need to do things in single-precision for this example to work:

In [ ]:
x = np.float32(1)
dx = np.float32(1e-8)

**Exercise 2:** start with one, and add one-millionth a million times. Compare accuracy of result using single- and double-precision


**Exercise 3:** test associativity of addition $a + b + c$ for $a = 12345678.0$, $b = -12345677.0$, $c = 0.5412382$. Try single and double precision.

**Exercise 4:** evaluate $\mathrm{sin}(n \pi)$ for $n =1, 10^{6}, 10^{12}, 10^{18}$

## 3. Example: instability of the quadratic formula

The roots of the quadratic equation $ax^2 + bx + c = 0$ are given by

$$x_1, x_2 = \frac{-b \pm \sqrt{b^2 - 4ac}}{2a}$$

The following is a naïve implementation:

In [ ]:
def quadratic(a,b,c,x):
    return a*x**2 + b*x + c

def root1(a,b,c):
    return (-b + np.sqrt(b*b - 4*a*c)) / (2*a)

def root2(a,b,c):
    return (-b - np.sqrt(b*b - 4*a*c)) / (2*a)

Consider a quadratic equation with roots $x_1 = \sqrt{3}$ and $x_2 = -\sqrt{2}\times 10^{-8}$:

In [ ]:
x1, x2 = np.sqrt(3), -np.sqrt(2)*1e-8

Define the corresponding $a$, $b$, $c$ and determine the fractional error in the calculated roots:

In [ ]:
a, b, c = 1.0, -(x1 + x2), x1*x2

In [ ]:
(root1(a,b,c) - x1)/x1

In [ ]:
(root2(a,b,c) - x2)/x2

The second root has suffered a loss of precision due to cancellation between terms in the formula:

In [ ]:
b, np.sqrt(b*b - 4*a*c)

We can circumvent this by noting that the roots satisfy $$x_2 = \frac{c}{a\, x_1}$$

In [ ]:
def root2p(a,b,c):
    return c/(a*root1(a,b,c))

In [ ]:
(root2p(a,b,c) - x2)/x2

The bottom line here is that care is sometimes needed to ensure that the algorithm being used does not incur loss of precision. For well-posed problems it is generally possible to find such algorithms.